In [1]:
import os
import glob
import numpy as np
import pandas as pd
import gc
import calendar
import datetime

import cudf
import cupy
import nvtabular as nvt
from merlin.dag import ColumnSelector
from merlin.schema import Schema, Tags
import nvtabular as nv

/home/ec2-user/SageMaker/.persisted_conda/transformers4rec-23.04/lib/python3.10/site-packages/merlin/dtypes/mappings/tf.py:52: UserWarning: Tensorflow dtype mappings did not load successfully due to an error: No module named 'tensorflow'
  warn(f"Tensorflow dtype mappings did not load successfully due to an error: {exc.msg}")
/home/ec2-user/SageMaker/.persisted_conda/transformers4rec-23.04/lib/python3.10/site-packages/merlin/dtypes/mappings/triton.py:53: UserWarning: Triton dtype mappings did not load successfully due to an error: No module named 'tritonclient'
  warn(f"Triton dtype mappings did not load successfully due to an error: {exc.msg}")


In [2]:
from nvtabular.ops import *


In [ ]:
mydata = cudf.read_parquet(f"s3a://zarklab-hive-test/buy-sell-updated/")


In [89]:
month1 = "202202"
month3 = "202204"
mydata = cudf.read_parquet(f"s3a://zarklab-token-recommender/buy-sell-updated-{month1}-{month3}")
zarklab-hive-test

# remove duplicate

In [90]:
mydata['timestamp'] = cudf.to_datetime(mydata['timestamp'])


In [91]:
print("Count with  repeated interactions: {}".format(len(mydata)))

# Sorts the dataframe by session and timestamp, to remove consecutive repetitions
# mydata.timestamp = mydata.timestamp.astype(int)
interactions_df = mydata.sort_values(['recipient', 'timestamp'])
past_ids = interactions_df['buyAsset'].shift(1).fillna()
session_past_ids = interactions_df['recipient'].shift(1).fillna()

# Keeping only no consecutive repeated in session interactions
interactions_df = interactions_df[~((interactions_df['recipient'] == session_past_ids) & (interactions_df['buyAsset'] == past_ids))]

print("Count after removed in-session repeated interactions: {}".format(len(interactions_df)))

Count with  repeated interactions: 1725084
Count after removed in-session repeated interactions: 955723


In [92]:
interactions_df.head()

,timestamp,recipient,txFee_eth,buyAsset,buyQty1,buyPrice,merge,combined_token_name,combined_token_symbol,token_category,token_rank_category,risky_flags,bluecheckmark
1664529,2022-03-12 06:41:35,0x000000000000000000000000000000000000005f,0.004351,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,0.00,0.07,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,Numeraire,nmr,Crowdfunding and Launchpads,Nano,0,TRUE
409331,2022-03-17 03:49:58,0x00000000000000000000000000000000000000e0,0.004352,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,0.62,17.64,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,Numeraire,nmr,Crowdfunding and Launchpads,Nano,0,TRUE
1302535,2022-03-13 10:02:30,0x00000000000000000000000000000000000000f1,0.004352,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,0.03,0.88,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,Numeraire,nmr,Crowdfunding and Launchpads,Nano,0,TRUE
963579,2022-03-10 23:21:45,0x0000000000000000000000000000000000000139,0.004353,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,0.00,0.12,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,Numeraire,nmr,Crowdfunding and Launchpads,Nano,0,TRUE
765950,2022-03-18 00:27:11,0x000000000000000000000000000000000000016e,0.004354,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,3.38,94.19,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,Numeraire,nmr,Crowdfunding and Launchpads,Nano,0,TRUE


In [93]:
dataset = nv.Dataset(interactions_df)

In [94]:
dataset.head()

,timestamp,recipient,txFee_eth,buyAsset,buyQty1,buyPrice,merge,combined_token_name,combined_token_symbol,token_category,token_rank_category,risky_flags,bluecheckmark
1664529,2022-03-12 06:41:35,0x000000000000000000000000000000000000005f,0.004351,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,0.00,0.07,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,Numeraire,nmr,Crowdfunding and Launchpads,Nano,0,TRUE
409331,2022-03-17 03:49:58,0x00000000000000000000000000000000000000e0,0.004352,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,0.62,17.64,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,Numeraire,nmr,Crowdfunding and Launchpads,Nano,0,TRUE
1302535,2022-03-13 10:02:30,0x00000000000000000000000000000000000000f1,0.004352,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,0.03,0.88,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,Numeraire,nmr,Crowdfunding and Launchpads,Nano,0,TRUE
963579,2022-03-10 23:21:45,0x0000000000000000000000000000000000000139,0.004353,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,0.00,0.12,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,Numeraire,nmr,Crowdfunding and Launchpads,Nano,0,TRUE
765950,2022-03-18 00:27:11,0x000000000000000000000000000000000000016e,0.004354,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,3.38,94.19,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,Numeraire,nmr,Crowdfunding and Launchpads,Nano,0,TRUE


In [95]:
SESSIONS_MAX_LENGTH = 20

# Encodes categorical features as contiguous integers
cat_feats = ColumnSelector(['buyAsset','token_category','token_rank_category','risky_flags']) >> nvt.ops.Categorify()



In [96]:
# create time features
session_ts = ColumnSelector(['timestamp'])
session_time = (
    session_ts >> 
    nvt.ops.LambdaOp(lambda col: cudf.to_datetime(col)) >> 
    nvt.ops.Rename(name = 'timestamp')
)
sessiontime_weekday = (
    session_time >> 
    nvt.ops.LambdaOp(lambda col: col.dt.weekday) >> 
    nvt.ops.Rename(name ='et_dayofweek')
)

In [97]:
# Derive cyclical features: Define a custom lambda function 
def get_cycled_feature_value_sin(col, max_value):
    value_scaled = (col + 0.000001) / max_value
    value_sin = np.sin(2*np.pi*value_scaled)
    return value_sin

weekday_sin = sessiontime_weekday >> (lambda col: get_cycled_feature_value_sin(col+1, 7)) >> nvt.ops.Rename(name = 'et_dayofweek_sin')


In [98]:
time_features = (
    session_time +
    sessiontime_weekday +
    weekday_sin )


In [99]:
txFee_eth_norm =  ColumnSelector(['txFee_eth']) >> nvt.ops.LogOp() >> nvt.ops.Normalize(out_dtype=np.float32) >> nvt.ops.Rename(name='txFee_eth_log_norm')
buyQty1_norm =  ColumnSelector(['buyQty1'])  >> nvt.ops.LogOp()>> nvt.ops.Normalize(out_dtype=np.float32) >> nvt.ops.Rename(name='buyQty1_log_norm')
buyPrice_norm =  ColumnSelector(['buyPrice']) >>nvt.ops.Clip(min_value=0, max_value=1000000)>> nvt.ops.LogOp() >> nvt.ops.Normalize(out_dtype=np.float32) >> nvt.ops.Rename(name='buyPrice_log_norm')


In [100]:
features = ColumnSelector(['recipient']) + cat_feats + time_features + txFee_eth_norm + buyQty1_norm + buyPrice_norm


In [101]:
# Define Groupby Operator
groupby_features = features >> nvt.ops.Groupby(
    groupby_cols=["recipient"], 
    sort_cols=["timestamp"],
    aggs={
        'buyAsset': ["list", "count"],
        'timestamp': ["first"],  
        'et_dayofweek_sin': ["list"],
        'txFee_eth_log_norm': ["list"],
        'buyQty1_log_norm': ["list"],
        'buyPrice_log_norm': ["list"],
        'token_category': ["list"],
        'token_rank_category': ["list"],
        'risky_flags': ["list"]
    },
    name_sep="-")

In [102]:
# Select and truncate the sequential features
sequence_features_truncated = (
    groupby_features['token_category-list','token_rank_category-list','risky_flags-list']
    >> nvt.ops.ListSlice(-SESSIONS_MAX_LENGTH) 
)

sequence_features_truncated_item = (
    groupby_features['buyAsset-list']
    >> nvt.ops.ListSlice(-SESSIONS_MAX_LENGTH) 
    >> TagAsItemID()
)  
sequence_features_truncated_cont = (
    groupby_features['et_dayofweek_sin-list', 'txFee_eth_log_norm-list','buyQty1_log_norm-list','buyPrice_log_norm-list'] 
    >> nvt.ops.ListSlice(-SESSIONS_MAX_LENGTH) 
    >> nvt.ops.AddMetadata(tags=[Tags.CONTINUOUS])
)


In [103]:
MINIMUM_SESSION_LENGTH = 2
selected_features = (
    groupby_features['buyAsset-count', 'timestamp-first', 'recipient'] + 
    sequence_features_truncated +
    sequence_features_truncated_item +
    sequence_features_truncated_cont
)
    
filtered_sessions = selected_features >> nvt.ops.Filter(f=lambda df: df["buyAsset-count"] >= MINIMUM_SESSION_LENGTH)

seq_feats_list = filtered_sessions['buyAsset-list', 'et_dayofweek_sin-list', 'txFee_eth_log_norm-list','buyQty1_log_norm-list','buyPrice_log_norm-list','token_category-list','token_rank_category-list','risky_flags-list'] >>  nvt.ops.ValueCount()

workflow = nvt.Workflow(filtered_sessions['recipient', 'timestamp-first'] + seq_feats_list)

In [104]:
workflow = nvt.Workflow.load('./preprocessWorkflow')
#workflow.fit(dataset)


In [85]:
out = workflow.transform(dataset)

In [86]:
out.head(30)

,recipient,timestamp-first,buyAsset-list,et_dayofweek_sin-list,txFee_eth_log_norm-list,buyQty1_log_norm-list,buyPrice_log_norm-list,token_category-list,token_rank_category-list,risky_flags-list
0,0x000000000004d7463d0f9c77383600bc82d612f5,2022-01-01 20:14:35,"[489, 418, 956, 24, 359, 1476, 1017, 714, 503,...","[0.43388293040961884, 0.43388293040961884, -0....","[-0.9175754, -0.83067816, -0.83143824, -0.8746...","[0.6312247, 0.47244197, 0.35492983, 0.55758375...","[0.23752865, 0.23259208, 0.23522705, 0.7562598...","[20, 28, 34, 4, 20, 13, 23, 13, 15, 4, 6, 4, 1...","[3, 3, 5, 3, 3, 3, 5, 5, 6, 3, 5, 3, 3, 5, 5, ...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, ..."
1,0x0000000000086013192a2c9a506742a43c916e1e,2022-01-09 07:44:40,"[4, 12, 150, 4]","[8.975979006501142e-07, 0.7818320421108522, 0....","[2.0464702, 0.92810464, 0.78188777, 5.520375]","[-1.0953704, -0.9503246, -0.2560672, -1.1121765]","[0.33380482, -0.24131858, -0.47064275, 0.18693...","[5, 4, 4, 5]","[3, 3, 5, 3]","[3, 3, 3, 3]"
2,0x000000000008e4db6a6194c6957df47e30970dc2,2022-02-23 18:48:34,"[4, 5, 113]","[0.43388293040961884, 0.43388293040961884, -0....","[1.1193919, -0.6704848, -0.7880669]","[-1.1768427, -0.7273375, -0.25471848]","[-0.79159683, -1.9954994, -0.9924365]","[5, 3, 5]","[3, 4, 5]","[3, 4, 3]"
3,0x00000000000cd56832ce5dfbcbff02e7ec639bc9,2022-01-28 21:47:38,"[614, 4]","[-0.9749281119157542, 8.975979006501142e-07]","[0.17984277, 0.0010065734]","[-0.5672601, -1.1894143]","[-0.91757536, -1.4771128]","[4, 5]","[5, 3]","[3, 3]"
4,0x00000000005ef87f8ca7014309ece7260bbcdaeb,2022-01-10 19:19:47,"[12, 3]","[0.7818320421108522, -0.7818309228245777]","[1.4960439, 1.3195738]","[-0.5672601, -0.375894]","[1.1722331, -0.70449436]","[4, 3]","[3, 4]","[3, 3]"
5,0x000000000088e0120f9e6652cc058aec07564f69,2022-02-27 22:21:30,"[3, 252]","[8.975979006501142e-07, 0.9749277124471076]","[-0.5149825, -0.6488472]","[-0.9772498, -1.1934248]","[-2.913314, -3.7022686]","[3, 18]","[4, 5]","[3, 3]"
6,0x00000000009a41862f3b2b0c688b7c0d1940511e,2022-01-05 21:58:02,"[7, 3, 115, 7]","[0.43388293040961884, -0.9749281119157542, -0....","[0.27445528, 1.6043956, 0.5207243, 0.5109864]","[0.104293786, -0.4080194, 0.6173917, 0.30022687]","[1.0587045, -0.8227892, 0.8926254, 1.7782758]","[3, 3, 6, 3]","[4, 4, 5, 4]","[3, 3, 3, 3]"
7,0x0000000002ce79aacd54227d2163ff3791338975,2022-03-28 01:09:48,"[8, 48]","[0.7818320421108522, 0.9749277124471076]","[-0.7984459, -0.7888265]","[-0.5541091, -0.22554801]","[-0.019073183, -0.15272635]","[10, 3]","[6, 7]","[3, 4]"
8,0x00000000047476dfaaf929c8c15fd1e4472e7cf6,2022-01-10 11:24:36,"[18, 977]","[0.7818320421108522, 0.43388293040961884]","[1.3680595, -0.72346777]","[-0.8197306, -0.24944623]","[0.29946572, -0.21152355]","[12, 4]","[5, 3]","[4, 3]"
9,0x000000000e3952882af02e7db0f3157bac7c6b51,2022-03-05 12:17:39,"[81, 1208]","[-0.7818309228245777, -0.7818309228245777]","[-0.5782119, -0.82306945]","[-1.1557566, -0.031170143]","[-3.0342171, -0.9103034]","[5, 4]","[4, 5]","[3, 3]"


In [87]:
workflow.save('./preprocessWorkflow')

In [88]:
out.to_parquet(f'./data/{month1}-{month3}/data.parquet')